In [2]:
import re
import os
import json
import pandas as pd

In [3]:

def merge_csv_files_from_folder(folder_path):
    # Lista para almacenar los DataFrames
    df_list = []

    # Recorrer todos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Obtener la ruta completa del archivo
            file_path = os.path.join(folder_path, filename)
            
            # Leer el archivo CSV y agregarlo a la lista de DataFrames
            df = pd.read_csv(file_path)
            df_list.append(df)

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("No se encontraron archivos CSV en la carpeta.")
        return pd.DataFrame()  

data = pd.read_csv("data_processed/CSV/test_part1.csv")
data_filtered = data[data["hasDetails"]]
print(data)
print(data_filtered)

             id                                                url   price  \
0      58155817  /mercedes-benz-clase-c-c-cabrio-220-d-2p-diese...   32499   
1      58594130  /mazda-mx-5-active-18-2p-gasolina-2006-en-ciud...   10600   
2      58548465  /kia-sorento-diesel-2005-en-malaga-58548465-co...    4500   
3      58594037  /opel-cabrio-14-t-ss-excellence-2p-gasolina-20...   12450   
4      58707917  /porsche-911-turbo-coupe-2p-gasolina-2016-en-b...  134900   
...         ...                                                ...     ...   
38428  56981011  /hyundai-ix35-2.0-crdi-136cv-tecno-4x2-5p-dies...   13199   
38429  58785088  /opel-zafira-diesel-2006-en-barcelona-58785088...    4750   
38430  56831130  /opel-vivaro-16cdti-70kw-95cv-expression-l2h1-...   16950   
38431  58302014  /volkswagen-polo-gasolina-2006-en-sevilla-5830...    2200   
38432  58702977  /toyota-c-hr-1.8-125h-advance-5p-electrico-/-h...   28100   

             km    year         color  cubicCapacity          b